In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
import re,string,unicodedata

import warnings

In [ ]:
import pandas as pd 
import snscrape.modules.twitter as sntwitter # interact with the data and tweet objects after scraping.
import numpy as np # numpy adds support for large multidimensional arrays and matrices and collection of high level mathematical functions.
import matplotlib.pyplot as plt # matplot for data viz
import seaborn as sns # for viz

import nltk #natural language toolkit- for identifying and tagging parts of speech found in the text of natural language like English.
from nltk.tokenize import word_tokenize # nltk.tokenize -  breaking down vast data into smaller chunks
from nltk.stem import WordNetLemmatizer # nltk.stem - to determine domain vocabularies in domain analysis.
from nltk.stem.porter import PorterStemmer

from bs4 import BeautifulSoup
import string
import re
import textblob
from textblob import TextBlob # textblob - sentiment analyzer
from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

porter = PorterStemmer()
# porter to remove 'es' etc from words
lemmatizer = WordNetLemmatizer() 
# lemmatizer to return base form of a word from dictionary

from wordcloud import ImageColorGenerator
from PIL import Image
import warnings
%matplotlib inline

# Data Extraction:

In [ ]:
query = '(#travel OR #travelling Of #tourism OR #ttot OR #TTOT OR #trip of #vacation OR #wanderlust OR #holiday) lang:en'
tweets=[]

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    tweets.append([tweet.date, tweet.id, tweet.user.username, tweet.sourceLabel, tweet.user.location,
                    tweet.content, tweet.likeCount, tweet.retweetCount, tweet.hashtags, tweet.mentionedUsers])
df = pd.DataFrame(tweets, columns=['Date', 'ID', 'username', 'source', 'location', 'tweet', 'num_of_likes',
                                   'num_of_retweet', 'hashtags', 'mentions'])
df.to_csv("travel_tweets_full.csv")


In [8]:
# loading the dataset:
data=pd.read_csv(r"C:\Users\Megha\jupyter\SYNOPSIS\travel_tweets_full.csv")
data=pd.DataFrame(data)
data.head()

,Unnamed: 0,Date,ID,username,source,location,tweet,num_of_likes,num_of_retweet,hashtags,mentions
0,0,2023-04-01 05:17:18+00:00,1.640000e+18,cliff_89,Twitter Web App,Singapore,BLOG LINK: https://t.co/0MmGQPVcGs\n\n#Palace ...,0.0,0.0,"['Palace', 'Shirvanshahs', 'Icherisheher', 'Ol...",NaN
1,1,2023-04-01 02:30:00+00:00,1.640000e+18,trusttravel84,Twitter Web App,Mumbai Maharashtra India,Uncover the Wonders of Turkey and create memor...,1.0,1.0,"['travel', 'traveling', 'vacation', 'visiting'...",NaN
2,2,2023-04-01 00:59:28+00:00,1.640000e+18,readtwtr,Revive Social App,Worldwide,Geneva Exploring the Best of it: What to See a...,0.0,0.0,"['bucketlist', 'explore', 'ttot', 'travelblogg...",NaN
3,3,2023-03-31 22:08:00+00:00,1.640000e+18,TRAVOH_travel,TweetDeck,Worldwide,"Discover The Historic City Of Lucca, Italy\n#T...",0.0,0.0,"['TRAVOH', 'Solespire', 'Italy', 'Italian', 'L...",NaN
4,4,2023-03-31 22:02:00+00:00,1.640000e+18,tipplertravel,Twitter Web App,"New York, NY",Vietnam is a pretty travel-worthy country. Thi...,0.0,0.0,"['vietnam', 'halongbay', 'unescoworldheritage'...",NaN


In [9]:
# total no. of tweets extracted
print('no. of tweets extracted: ',data.tweet.count())

no. of tweets extracted:  81705


# Data Cleaning and Preprocessing:

### Removing null values:

In [10]:
print('no. of rows with empty cells in tweet column: ',data.tweet.isnull().sum())
data = data.dropna(subset=['tweet'])

no. of rows with empty cells in tweet column:  7


### Converting to lower case:

In [11]:
def convert_lowercase(column):
    column = column.str.lower()
    return column

data['processed_clean_tweets'] = convert_lowercase(data['tweet']) # creating new column to store clean and processed tweets

### Cleaning and pre-processing Tweets:

In [12]:
stop = stopwords.words('english')
wl = WordNetLemmatizer()
mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
           "'cause": "because", "could've": "could have", "couldn't": "could not", 
           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
           "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
           "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
           "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
           "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have",
           "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
           "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
           "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
           "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
           "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
           "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
           "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
           "she's": "she is", "should've": "should have", "shouldn't": "should not", 
           "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
           "that'd": "that would", "that'd've": "that would have", "that's": "that is", 
           "there'd": "there would", "there'd've": "there would have", "there's": "there is", 
           "here's": "here is","they'd": "they would", "they'd've": "they would have", 
           "they'll": "they will", "they'll've": "they will have", "they're": "they are", 
           "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
           "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
           "we're": "we are", "we've": "we have", "weren't": "were not", 
           "what'll": "what will", "what'll've": "what will have","what're": "what are",  
           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", 
           "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
           "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
           "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
           "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
           "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
           "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
           "you're": "you are", "you've": "you have" }

In [13]:
# creating a function t clean and process tweets
def clean_process_text(text,lemmatize = True):
    soup = BeautifulSoup(text, "html.parser") #remove html tags
    text = soup.get_text()
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")]) #expanding chatwords and contracts clearing contractions
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_clean.sub(r'',text)
    text = re.sub(r'\.(?=\S)', '. ',text) #add space after full stop
    text = re.sub(r'http\S+', '', text) #remove urls
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #remove punctuation
    #tokens = re.split('\W+', text) #create tokens
    if lemmatize:
        text = " ".join([wl.lemmatize(word) for word in text.split() if word not in stop and word.isalpha()]) #lemmatize
    else:
        text = " ".join([word for word in text.split() if word not in stop and word.isalpha()]) 
    return text

In [14]:
# creating a copy of original data:
data_copy = data.copy()

In [15]:
data['processed_clean_tweets'] = data['processed_clean_tweets'].astype(str)
data['processed_clean_tweets'] = data['processed_clean_tweets'].apply(clean_process_text,lemmatize = True)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [16]:
tweets_string = data['processed_clean_tweets'].tolist()
tweets_string = " ".join(tweets_string) # convert the tweet text into a string separate with " "

In [19]:
data.head()

,Unnamed: 0,Date,ID,username,source,location,tweet,num_of_likes,num_of_retweet,hashtags,mentions,processed_clean_tweets
0,0,2023-04-01 05:17:18+00:00,1.640000e+18,cliff_89,Twitter Web App,Singapore,BLOG LINK: https://t.co/0MmGQPVcGs\n\n#Palace ...,0.0,0.0,"['Palace', 'Shirvanshahs', 'Icherisheher', 'Ol...",NaN,blog link palace shirvanshahs within icherishe...
1,1,2023-04-01 02:30:00+00:00,1.640000e+18,trusttravel84,Twitter Web App,Mumbai Maharashtra India,Uncover the Wonders of Turkey and create memor...,1.0,1.0,"['travel', 'traveling', 'vacation', 'visiting'...",NaN,uncover wonder turkey create memory last lifet...
2,2,2023-04-01 00:59:28+00:00,1.640000e+18,readtwtr,Revive Social App,Worldwide,Geneva Exploring the Best of it: What to See a...,0.0,0.0,"['bucketlist', 'explore', 'ttot', 'travelblogg...",NaN,geneva exploring best see adventure bucketlist...
3,3,2023-03-31 22:08:00+00:00,1.640000e+18,TRAVOH_travel,TweetDeck,Worldwide,"Discover The Historic City Of Lucca, Italy\n#T...",0.0,0.0,"['TRAVOH', 'Solespire', 'Italy', 'Italian', 'L...",NaN,discover historic city lucca italy travoh sole...
4,4,2023-03-31 22:02:00+00:00,1.640000e+18,tipplertravel,Twitter Web App,"New York, NY",Vietnam is a pretty travel-worthy country. Thi...,0.0,0.0,"['vietnam', 'halongbay', 'unescoworldheritage'...",NaN,vietnam pretty travelworthy country ha long ba...


In [21]:
print('no. of preprocessed tweets after cleaning and preprocessing: ',data.processed_clean_tweets.count())

no. of preprocessed tweets after cleaning and preprocessing:  81705


data.to_csv('proccess_clean_syn.csv')